# INTRODUCTION


**"Automatically extracting personal information from Passport document".**

Attributes to be extracted from each passport record:

First Name (Given Name)

Last Name (Surname)

Date of Birth.

## Organization

> [Find Data Paths](#scrollTo=5gs_vOz4l5_m)

> [Processing Data and Extracting Text](#scrollTo=FWVRPpNVmS1H)

> [Time Elapsed for data Extraction](#scrollTo=sycN9ZJWc2g4)

> [Helper functions](#scrollTo=GgpJhIQdkOiM)

> [Working With Training Data](#scrollTo=gtMt60iUgKU3)

> [Working with Test Data](#scrollTo=Q0UyWn7pmhNc)

> [Results on Validation data](#scrollTo=LYync1q8m467)

-Submitted By: Apoorva Rastogi

In [ ]:
#Importing Necessary Libraries
%matplotlib inline
from google.colab import files
import os
import zipfile
import gdown
import difflib
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from skimage.io import imread
import cv2
from PIL import Image
import sys

import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
#import tesseract
# Before Importing pytesseract Kindly insure that you have it installed as well as tesseract-ocr

# The following commands worked for me
!sudo apt install tesseract-ocr
!pip3 install pytesseract

import pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,181 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
# Importing Data
#from google.colab import drive
#drive.mount('/content/drive')

# I am using gdown to get data as the dataset is small and I could get data directly from the link.
# If your data is in the drive the above lines could be used to mount the drive and access the files.


!gdown --folder https://drive.google.com/drive/folders/1isrONXBvsIRbFOlNcXukQsVYvZKzMsIP

Retrieving folder list
Retrieving folder 1ifsllj9Hg415P4pO6jWDQc_a2ifR_yFa Test Data
Processing file 1n2JTRUF3ZcQHAXvuWkON65irxjjfv-iB Passport_Sample15.png
Processing file 1n26gKFCI3isteHewIkwKt49UwT5lYrQn Passport_Sample16.png
Processing file 1n0YKK0PB17RrXJvCqRc6ztMs3E-RsARo Passport_Sample17.png
Processing file 1mwFubtRBvHeyz3FdrPjeD2Jf4atoy5Ux Passport_Sample18.png
Processing file 1mqYbsi5Q_coV7nxXMY69ocHyKxVPDO0V Passport_Sample19.png
Processing file 1mo6W1umRwryOoqIleg-05yvRtvbVNKJL Passport_Sample20.png
Processing file 1n2ocCQKz3iTrAliIDv4EsWmRuGFCg-Mm Passport_Sample21.png
Processing file 1rjqXmLDY15r4ICuMjP8YX0CA_si1xaAO Test Data - Ground Truth.xlsx
Retrieving folder 1ictxPMIqwXmstxQvL57-LZgoOnzsUS_b Training Data
Processing file 1R_akIBiujm8u37XoGNHFOXdOl4qIxXOL Passport_Sample0.png
Processing file 1JAPWZcEO8mSD8_BsxsnwE0UKHkJFGkek Passport_Sample1.png
Processing file 1rLvH2T02UdhY6zaZs6DL787m5huSBLY2 Passport_Sample2.png
Processing file 1jfeUe_wLfG6RsOkS-hdwZEKaeLLa8zWy Pa

# Find Data Paths for all Data




In [ ]:
# We need to get all the paths for the images to later load them
imagepathsTrain = []
imagepathsTest = []
imagepathsVal = []
# Go through all the files and subdirectories inside a folder and save path to images inside list
for root, dirs, files in os.walk("/content/Passport", topdown=False):
  for name in files:
    path = os.path.join(root, name)
    if path.endswith(tuple(["png","jpg"])): # Observing the dataset we only have the two file types.
      if ("Training Data" in path) : # We want only the specified images
           imagepathsTrain.append(path)
      if ("Test Data" in path) :
           imagepathsTest.append(path)
      if ("Validation Data" in path) :
           imagepathsVal.append(path)

print(len(imagepathsTrain)) # If > 0, then an image was loaded
print(len(imagepathsTest))
print(len(imagepathsVal))

22
7
7


# Processing Data and Extracting Text:

### **Methods Used for Image Processing:**

**1: GrayScaling:** Here, we use OpenCV to read the image and convert it to grayscale. Grayscale images often work better for OCR because they simplify the image and remove color noise. Depending on the specific quality issues of the image, you may need to apply additional pre-processing techniques to improve the OCR results, such as image filtering or denoising.



 **2: Thresholding:** Thresholding is a common image processing technique used to convert grayscale images into binary images. It helps to segment the image into regions of interest based on pixel intensity. The basic idea is to set a threshold value, and all pixels with intensity values above the threshold are set to one color (e.g., white), while all pixels with intensity values below the threshold are set to another color (e.g., black).

Thresholding is useful in preprocessing images to simplify and extract important information, such as text, objects, or regions of interest. It can be used to remove noise, enhance features, or prepare the image for further analysis.[ Documentation on Thresholding](https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html)

### **Library Used for Text Extraction:**

Tesseract is an open-source OCR engine developed by Google. It can be used with Python using the pytesseract library.

Before proceeding, make sure you have installed Tesseract OCR on your system. You can download it from the official GitHub repository: [tesseract-ocr]( https://github.com/tesseract-ocr/tesseract)

Once Tesseract is installed, you can use the pytesseract library in Python:

The installation methods mentioned above while imporitng libraries worked for me.

In [ ]:
def getData(imagepaths): # Function Gets Data from the input Path

  X =[]

  for path in imagepaths:
    img = cv2.imread(path) # Reads image and returns np.array
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converts into the corret colorspace (GRAY)
    ret,thresh1 = cv2.threshold(gray_image,0,255,cv2.THRESH_BINARY) # Thresholding to Improve data Extraction
    curr=[]

    # Extracting Text
    text =pytesseract.image_to_string(thresh1)

    # Extracting File Name
    fileName = path.split('/')[4]
    dot_position = fileName.find(".")
    fileName = fileName[:dot_position]
    curr.append(fileName)

    # Processing Extracted Text
    text = text.splitlines()
    text = [string for string in text if string.strip() != ""]
    # Skip if no text extracted
    Text1 = text[0] if 1 < len(text) else None

    if(Text1):
      result = text[1].split(' ')[1] if 1 < len(text[1].split(' ')) else text[1]
      curr.append(result)
      result = text[2]
      curr.append(result)
      result = text[3]
      curr.append(result)
    else:
      curr.append('not clear')
      curr.append('not clear')
      curr.append('01Jul1997')

    X.append(curr)
  return X

#XTest = getData(imagepathsTest)
#XTrain = getData(imagepathsTrain)
#XVal  = getData(imagepathsVal)

In [ ]:
#Time Elapsed for data Extraction

import time

# Start time
start_time = time.time()

# Your code block to be measured
XTrain = getData(imagepathsTrain)

# End time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print("Time taken to get Training Data Values:", elapsed_time, "seconds")


# Start time
start_time = time.time()

# Your code block to be measured
XTest = getData(imagepathsTest)

# End time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print("Time taken to get Test Data Values:", elapsed_time, "seconds")


# Start time
start_time = time.time()

# Your code block to be measured
XVal = getData(imagepathsVal)

# End time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print("Time taken to get Validation Data Values:", elapsed_time, "seconds")


Time taken to get Training Data Values: 17.938997745513916 seconds
Time taken to get Test Data Values: 7.415744304656982 seconds
Time taken to get Validation Data Values: 3.29471755027771 seconds


In [ ]:
#Helper functions

def calculate_character_matching_percentage(string1, string2):
    len_longer = max(len(string1), len(string2))

    common_chars = sum(c1 == c2 for c1, c2 in zip(string1, string2))

    matching_percentage = (common_chars / len_longer) * 100
    return matching_percentage

from datetime import datetime

def detect_date_format(date_str):
    formats_to_try = [
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d",
        "%m/%d/%Y",
        "%d/%m/%Y",
        "%m-%d-%Y",
        "%d-%m-%Y",
    ]

    for date_format in formats_to_try:
        try:
            parsed_date = datetime.strptime(date_str, date_format)
            return date_format
        except ValueError:
            pass

    return None  # If the format couldn't be detected


def convert_date_format(input_date):
  try:
    # Parse the input date string into a datetime object
    format = detect_date_format(input_date)
    if(format):
      datetime_obj = datetime.strptime(input_date,format)# '%Y-%m-%d %H:%M:%S')
    else:
      datetime_obj = datetime.strptime(input_date,'%Y-%m-%d %H:%M:%S')

    # Format the datetime object into the desired format
    formatted_date = datetime_obj.strftime('%d %b %Y')

    return formatted_date
  except ValueError:
        # Handle cases where the input does not match the expected format
        return input_date

def edit_distance(str1, str2):
    m, n = len(str1), len(str2)

    # Create a 2D array to store the edit distances
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize the base case values
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Calculate the edit distance using dynamic programming
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[m][n]



# Working With Training Data

The full code Uses many similar code snippets repeated and thus run them one by one or else the results would not match. The code can be mode more modular and robust but for now it is deals with the limited data available.

Accuracy of information extraction: To evaluate the model's performance in correctly extracting relevant information from the ID documents we use the below metrics:



1.   Character Accuracy: The percentage of correctly recognized characters.
2.   Word Accuracy: The percentage of correctly recognized words.
3.   
Edit Distance: Measures the difference between predicted text and ground truth in terms of the number of insertions, deletions, and substitutions required to match the text.



In [ ]:
#Extracting Required Training Data
TrainAns = pd.read_excel("/content/Passport/Training Data/Train Data - Ground Truth.xlsx")
Data = TrainAns[['File Name','First Name /Given Name','Last Name / Surname','Date of Birth']]
Data['File Name'] = Data['File Name'].replace('_', '', regex=True)

<ipython-input-9-3aca971e5e74>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data['File Name'] = Data['File Name'].replace('_', '', regex=True)


In [ ]:
# Formatting data For comparison
df = pd.DataFrame(XTrain, columns = ['File Name','Last Name / Surname','First Name /Given Name','Date of Birth'])
df['File Name'] = df['File Name'].replace('_', '', regex=True)
#with pd.ExcelWriter('/content/Passport/extractedTrainingData.xlsx') as writer:
#   df.to_excel(writer, sheet_name='Sheet1', index=False)

In [ ]:
lastNameWordTrainMatch = 0
lastNameCharTrainMatch = 0
noOfChar = 0
editDist = 0
noOfRecords = len(df['File Name'])

for i in range(len(df['File Name'])):
  searchtext = df['File Name'][i]
  #finding concerned row
  concernedRow = Data[Data['File Name'].str.contains(searchtext,case=False, na=False)]

  #final strings
  string1 = str(concernedRow['Last Name / Surname'].iloc[0])
  string2 = str(df['Last Name / Surname'][i])
  editDist = editDist + edit_distance(string1,string2)
  # string comparison
  percentage = calculate_character_matching_percentage(string1, string2)

  if (percentage == 100):
    lastNameWordTrainMatch = lastNameWordTrainMatch + 1

  lastNameCharTrainMatch = lastNameCharTrainMatch + (percentage * len(string1))/100
  noOfChar = noOfChar+len(string1)


print(f"Percentage of Words matching in Training Data for Last Name: {lastNameWordTrainMatch/noOfRecords*100:.2f}%")
print(f"Percentage of Char matching in Training Data for Last Name: {lastNameCharTrainMatch/noOfChar*100:.2f}%")
print("Edit Distance combined for all Last name in Training Data :", editDist)


Percentage of Words matching in Training Data for Last Name: 72.73%
Percentage of Char matching in Training Data for Last Name: 68.94%
Edit Distance combined for all Last name in Training Data : 54


In [ ]:
firstNameWordTrainMatch = 0
firstNameCharTrainMatch = 0
noOfChar = 0
editDist = 0
noOfRecords = len(df['File Name'])
for i in range(len(df['File Name'])):
  searchtext = df['File Name'][i]
  #finding concerned row
  concernedRow = Data[Data['File Name'].str.contains(searchtext,case=False, na=False)]

  #final strings
  string1 = str(concernedRow['First Name /Given Name'].iloc[0])
  string2 = str(df['First Name /Given Name'][i])
  editDist = editDist + edit_distance(string1,string2)

  percentage = calculate_character_matching_percentage(string1, string2)
  if (percentage == 100):
    firstNameWordTrainMatch = firstNameWordTrainMatch + 1

  firstNameCharTrainMatch = firstNameCharTrainMatch + (percentage * len(string1))/100
  noOfChar = noOfChar+len(string1)

print(f"Percentage of Words matching in Training Data for First Name: {firstNameWordTrainMatch/noOfRecords*100:.2f}%")
print(f"Percentage of Char matching in Training Data for First Name: {firstNameCharTrainMatch/noOfChar*100:.2f}%")
print("Edit Distance combined for all First name in Training Data :", editDist)


Percentage of Words matching in Training Data for First Name: 63.64%
Percentage of Char matching in Training Data for First Name: 67.86%
Edit Distance combined for all First name in Training Data : 56


In [ ]:
dobWordTrainMatch = 0
dobCharTrainMatch = 0
noOfChar = 0
editDist = 0
noOfRecords = len(df['File Name'])

for i in range(len(df['File Name'])):
  searchtext = df['File Name'][i]
  #finding concerned row
  concernedRow = Data[Data['File Name'].str.contains(searchtext,case=False, na=False)]

  #final strings
  string1 = str(concernedRow['Date of Birth'].iloc[0])
  output_date_str = convert_date_format(string1)
  string2 = str(df['Date of Birth'][i])
  editDist = editDist + edit_distance(output_date_str,string2)

  percentage = calculate_character_matching_percentage(output_date_str, string2)

  if (percentage == 100):
    dobWordTrainMatch = dobWordTrainMatch + 1

  dobCharTrainMatch = dobCharTrainMatch + (percentage * len(string1))/100
  noOfChar = noOfChar+len(string1)


print(f"Percentage of Words matching in Training Data for DOB: {dobWordTrainMatch/noOfRecords*100:.2f}%")
print(f"Percentage of Char matching in Training Data for DOB: {dobCharTrainMatch/noOfChar*100:.2f}%")
print("Edit Distance combined for all DOB in Training Data :", editDist)


Percentage of Words matching in Training Data for DOB: 45.45%
Percentage of Char matching in Training Data for DOB: 63.64%
Edit Distance combined for all DOB in Training Data : 70


# Working with Test Data


As we are using a pretrained model we find the same metric for testing data as well
Accuracy of information extraction: To evaluate the model's performance in correctly extracting relevant information from the ID documents we use the below metrics:



1.   Character Accuracy: The percentage of correctly recognized characters.
2.   Word Accuracy: The percentage of correctly recognized words.
3.   
Edit Distance: Measures the difference between predicted text and ground truth in terms of the number of insertions, deletions, and substitutions required to match the text.


In [ ]:
#Extracting Required Test Data
TrainAns = pd.read_excel("/content/Passport/Test Data/Test Data - Ground Truth.xlsx")
Data = TrainAns[['File Name','First Name /Given Name','Last Name / Surname','Date of Birth']]
Data['File Name'] = Data['File Name'].replace('_', '', regex=True)

<ipython-input-14-aadddeb58384>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data['File Name'] = Data['File Name'].replace('_', '', regex=True)


In [ ]:
# Formatting data For comparison
df = pd.DataFrame(XTest, columns = ['File Name','Last Name / Surname','First Name /Given Name','Date of Birth'])
df['File Name'] = df['File Name'].replace('_', '', regex=True)

with pd.ExcelWriter('/content/Passport/extractedTestData.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)


In [ ]:
lastNameWordTestMatch = 0
lastNameCharTestMatch = 0
noOfChar = 0
editDist = 0
noOfRecords = len(df['File Name'])

for i in range(len(df['File Name'])):
  searchtext = df['File Name'][i]
  #finding concerned row
  concernedRow = Data[Data['File Name'].str.contains(searchtext,case=False, na=False)]

  string1 = str(concernedRow['Last Name / Surname'].iloc[0])
  string2 = str(df['Last Name / Surname'][i])
  editDist = editDist + edit_distance(string1,string2)

  percentage = calculate_character_matching_percentage(string1, string2)

  if (percentage == 100):
    lastNameWordTestMatch = lastNameWordTestMatch + 1

  lastNameCharTestMatch = lastNameCharTestMatch + (percentage * len(string1))/100
  noOfChar = noOfChar+len(string1)

#print(f"Percentage of character matching: {percentage:.2f}%")


print(f"Percentage of Words matching in Test Data for Last Name: {lastNameWordTestMatch/noOfRecords*100:.2f}%")
print(f"Percentage of Char matching in Test Data for Last Name: {lastNameCharTestMatch/noOfChar*100:.2f}%")
print("Edit Distance combined for all Last name in Test Data :", editDist)
# This should be 100 % but the models is not matching for PassportSample16 and on further observation
# The Ground Truth is Incorrect.


Percentage of Words matching in Test Data for Last Name: 85.71%
Percentage of Char matching in Test Data for Last Name: 85.71%
Edit Distance combined for all Last name in Test Data : 6


In [ ]:
firstNameWordTestMatch = 0
firstNameCharTestMatch = 0
noOfChar = 0
editDist = 0
noOfRecords = len(df['File Name'])
for i in range(len(df['File Name'])):
  searchtext = df['File Name'][i]
  concernedRow = Data[Data['File Name'].str.contains(searchtext,case=False, na=False)]


  string1 = str(concernedRow['First Name /Given Name'].iloc[0])
  string2 = str(df['First Name /Given Name'][i])
  editDist = editDist + edit_distance(string1,string2)

  percentage = calculate_character_matching_percentage(string1, string2)

  if (percentage == 100):
    firstNameWordTestMatch = firstNameWordTestMatch + 1

  firstNameCharTestMatch = firstNameCharTestMatch + (percentage * len(string1))/100
  noOfChar = noOfChar+len(string1)

#print(f"Percentage of character matching: {percentage:.2f}%")


print(f"Percentage of Words matching in Test Data for First Name: {firstNameWordTestMatch/noOfRecords*100:.2f}%")
print(f"Percentage of Char matching in Test Data for First Name: {firstNameCharTestMatch/noOfChar*100:.2f}%")
print("Edit Distance combined for all First name in Test Data :", editDist)

Percentage of Words matching in Test Data for First Name: 85.71%
Percentage of Char matching in Test Data for First Name: 82.05%
Edit Distance combined for all First name in Test Data : 6


In [ ]:
dobWordTestMatch = 0
dobCharTestMatch = 0
noOfChar = 0
editDist = 0
noOfRecords = len(df['File Name'])

for i in range(len(df['File Name'])):
  searchtext = df['File Name'][i]
  concernedRow = Data[Data['File Name'].str.contains(searchtext,case=False, na=False)]


  string1 = str(concernedRow['Date of Birth'].iloc[0])
  output_date_str = convert_date_format(string1)
  string2 = str(df['Date of Birth'][i])
  editDist = editDist + edit_distance(output_date_str,string2)

  percentage = calculate_character_matching_percentage(output_date_str, string2)

  if (percentage == 100):
    dobWordTestMatch = dobWordTestMatch + 1

  dobCharTestMatch = dobCharTestMatch + (percentage * len(string1))/100
  noOfChar = noOfChar+len(string1)


print(f"Percentage of Words matching in Test Data for DOB: {dobWordTestMatch/noOfRecords*100:.2f}%")
print(f"Percentage of Char matching in Test Data for DOB: {dobCharTestMatch/noOfChar*100:.2f}%")
print("Edit Distance combined for all DOB in Test Data :", editDist)

Percentage of Words matching in Test Data for DOB: 71.43%
Percentage of Char matching in Test Data for DOB: 84.42%
Edit Distance combined for all DOB in Test Data : 4


In [ ]:
# Results on Validation data

#Since we do not have final solution for Validation Set We could not Match it
# Observing below we can see that only one data is extracted from the images present
df = pd.DataFrame(XVal, columns = ['File Name','Last Name / Surname','First Name /Given Name','Date of Birth'])
df['File Name'] = df['File Name'].replace('_', '', regex=True)

with pd.ExcelWriter('/content/Passport/extractedValData.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)

# Further Work


Explore ways to get information out of old and dark images.

# NOT PART OF SUBMISSION

Trails works to get information from old and dark Images. Need to study topic indepth for better solution

In [ ]:
import cv2
# Additional Trails to extract data from not clear Images
# Need more time to work this out and try other methods for this task
def enhance_image(input_image_path):
    # Read the image
    #image = cv2.imread(input_image_path)
    image = input_image_path

    # Check if the image is loaded successfully
    if image is None:
        print("Error: Unable to load the image.")
        return

    # Apply image filtering (e.g., using a bilateral filter)
    filtered_image = cv2.bilateralFilter(image, d=1, sigmaColor=50, sigmaSpace=50)

    return filtered_image

# Example usage:
input_image_path = "/content/Passport/Validation Data/Passport_Sample34.jpg"
#enhance_image(input_image_path)


#img = cv2.imread("/content/Passport/Validation Data/Passport_Sample34.jpg")
img = cv2.imread('/content/Passport/Validation Data/Passport_Sample34.jpg', cv2.IMREAD_GRAYSCALE)
#gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,thresh1 = cv2.threshold(gray_image,160,255,cv2.THRESH_BINARY)
#cv2_imshow(thresh1 )
myimg = enhance_image(thresh1)
#cv2_imshow(myimg)
#print(pytesseract.image_to_string(myimg))


In [ ]:
#!pip3 install pytesseract

In [ ]:
XVal = [] # Image data


for path in imagepathsVal:
    img = cv2.imread(path) # Reads image and returns np.array
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converts into the corret colorspace (GRAY)
    ret,thresh1 = cv2.threshold(gray_image,150,255,cv2.THRESH_BINARY) # Thresholding to Improve data Extraction
    thresholded_img = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                           cv2.THRESH_BINARY, 11, 2)
    curr=[]

    # Extracting Text
    text =pytesseract.image_to_string(thresholded_img)
    #print(text)

    # Extracting File Name
    fileName = path.split('/')[4]
    dot_position = fileName.find(".")
    fileName = fileName[:dot_position]
    curr.append(fileName)

    #print(text)
    #cv2_imshow(thresholded_img)

    # Processing Extracted Text
    text = text.splitlines()
    text = [string for string in text if string.strip() != ""]
    # Skip if no text extracted
    Text1 = text[0] if 1 < len(text) else None

''' if(Text1):
      result = text[1].split(' ')[1] if 1 < len(text[1].split(' ')) else text[1]
      curr.append(result)
      result = text[2]
      curr.append(result)
      result = text[3]
      curr.append(result)
    else:
      curr.append('not clear')
      curr.append('not clear')
      curr.append('01Jul1997')

    XVal.append(curr)'''

" if(Text1):\n      result = text[1].split(' ')[1] if 1 < len(text[1].split(' ')) else text[1]\n      curr.append(result)\n      result = text[2]\n      curr.append(result)\n      result = text[3]\n      curr.append(result)\n    else:\n      curr.append('not clear')\n      curr.append('not clear')\n      curr.append('01Jul1997')\n\n    XVal.append(curr)"

In [ ]:
!pip3 install keras-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 60.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.3 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19580 sha256=d9415cb2d8bb3a1f13447716a64fe998b7e66a2dde77f4d854ad18d3955a3774
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [ ]:
import matplotlib.pyplot as plt

import keras_ocr

# keras-ocr will automatically download pretrained
# weights for the detector and recognizer.
pipeline = keras_ocr.pipeline.Pipeline()

# Get a set of three example images
#images = [
 #   keras_ocr.tools.read(url) for url in [
 #       'https://upload.wikimedia.org/wikipedia/commons/b/bd/Army_Reserves_Recruitment_Banner_MOD_45156284.jpg',
 #       'https://upload.wikimedia.org/wikipedia/commons/e/e8/FseeG2QeLXo.jpg',
 #       'https://upload.wikimedia.org/wikipedia/commons/b/b4/EUBanana-500x112.jpg'
 #   ]
#]
for path in imagepathsVal:
    img = cv2.imread(path) # Reads image and returns np.array
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converts into the corret colorspace (GRAY)
    ret,thresh1 = cv2.threshold(gray_image,150,255,cv2.THRESH_BINARY) # Thresholding to Improve data Extraction
    curr=[]
# Each list of predictions in prediction_groups is a list of
# (word, box) tuples.
#prediction_groups = pipeline.recognize(thresh1)

# Plot the predictions
#fig, axs = plt.subplots(nrows=len(img), figsize=(20, 20))
#for ax, image, predictions in zip(axs, img, prediction_groups):
#    keras_ocr.tools.drawAnnotations(image=image, predictions=predictions, ax=ax)

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
